In [2]:
%load_ext autoreload
%autoreload 2

In [41]:
# To ensure our src module can be found and imported
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import os
import sys
import subprocess
import pandas as pd

from datetime import datetime, timedelta
from pathlib import Path

    
from src.data.binance_downloader import download_historical_daily_klines

In [19]:
Path.cwd().parent

PosixPath('/Users/jonathanlim/workspace/personal/bitcoin-god')

In [63]:
DATA_DIR = Path.cwd().parent / 'data'
RAW_DATA_DIR = DATA_DIR / 'raw'
PROCESSED_DATA_DIR = DATA_DIR / 'processed'

BINANCE_HISTORICAL_DATA_DIR = RAW_DATA_DIR / 'binance_historical'
BINANCE_HISTORICAL_FILES_DIR = BINANCE_HISTORICAL_DATA_DIR / 'data/spot/daily/klines/BTCUSDT/1m'
BINANCE_HISTORICAL_DF_PATH = PROCESSED_DATA_DIR / 'binance_historical_df.csv'

TRADING_TYPE = 'spot'
TICKER_SYMBOLS = ['BTCUSDT']
INTERVALS = ['1m']
# No available data before 2021-03-01
START_DATE = '2021-03-01'
END_DATE = (datetime.now() - timedelta(days=1) ).strftime('%Y-%m-%d')
# Reference: https://github.com/binance/binance-public-data/tree/master
RAW_DF_HEADERS = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'num_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']

# Ensure directories are present
BINANCE_HISTORICAL_DATA_DIR.mkdir(parents=True, exist_ok=True)

In [39]:
download_historical_daily_klines(TRADING_TYPE, 
                                 TICKER_SYMBOLS, 
                                 len(TICKER_SYMBOLS), 
                                 INTERVALS, 
                                 START_DATE, 
                                 END_DATE, 
                                 str(BINANCE_HISTORICAL_DATA_DIR))

Found 1 symbols
[1/1] - start download daily BTCUSDT klines 

file already exists! /Users/jonathanlim/workspace/personal/bitcoin-god/data/raw/binance_historical/data/spot/daily/klines/BTCUSDT/1m/BTCUSDT-1m-2021-03-01.zip

file already exists! /Users/jonathanlim/workspace/personal/bitcoin-god/data/raw/binance_historical/data/spot/daily/klines/BTCUSDT/1m/BTCUSDT-1m-2021-03-02.zip

file already exists! /Users/jonathanlim/workspace/personal/bitcoin-god/data/raw/binance_historical/data/spot/daily/klines/BTCUSDT/1m/BTCUSDT-1m-2021-03-03.zip

file already exists! /Users/jonathanlim/workspace/personal/bitcoin-god/data/raw/binance_historical/data/spot/daily/klines/BTCUSDT/1m/BTCUSDT-1m-2021-03-04.zip

file already exists! /Users/jonathanlim/workspace/personal/bitcoin-god/data/raw/binance_historical/data/spot/daily/klines/BTCUSDT/1m/BTCUSDT-1m-2021-03-05.zip

file already exists! /Users/jonathanlim/workspace/personal/bitcoin-god/data/raw/binance_historical/data/spot/daily/klines/BTCUSDT/1m/BTCUS

In [42]:
df = pd.read_csv('/Users/jonathanlim/workspace/personal/bitcoin-god/data/raw/binance_historical/data/spot/daily/klines/BTCUSDT/1m/BTCUSDT-1m-2023-01-07.zip')

In [50]:
# Read all files in BINANCE_HISTORICAL_FILES_DIR
files = sorted([str(path) for path in BINANCE_HISTORICAL_FILES_DIR.glob('**/*') if path.is_file()])

In [59]:
df_list = [pd.read_csv(path, names=RAW_DF_HEADERS) for path in files]
historical_df = pd.concat(df_list, axis=0, ignore_index=True)

In [61]:
historical_df

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,1614556800000,45134.11,45266.77,45130.34,45260.74,72.517978,1614556859999,3.277691e+06,2207,33.689150,1.522869e+06,0
1,1614556860000,45252.67,45362.07,45250.64,45356.00,65.371778,1614556919999,2.961835e+06,2028,32.499895,1.472609e+06,0
2,1614556920000,45356.00,45371.41,45104.36,45128.57,128.114624,1614556979999,5.795551e+06,2706,47.268294,2.138666e+06,0
3,1614556980000,45128.57,45194.65,45020.87,45037.36,59.964922,1614557039999,2.706678e+06,1502,25.519749,1.152231e+06,0
4,1614557040000,45036.62,45107.01,44977.82,45032.48,57.852895,1614557099999,2.605675e+06,1250,22.489983,1.013207e+06,0
...,...,...,...,...,...,...,...,...,...,...,...,...
975401,1673135700000,16945.03,16945.26,16944.72,16945.11,25.793170,1673135759999,4.370653e+05,1119,13.070310,2.214776e+05,0
975402,1673135760000,16944.97,16945.72,16944.64,16945.51,47.295250,1673135819999,8.014295e+05,1273,34.357750,5.822022e+05,0
975403,1673135820000,16945.23,16945.66,16944.29,16945.11,21.812910,1673135879999,3.696213e+05,1085,10.219850,1.731761e+05,0
975404,1673135880000,16945.23,16945.64,16943.73,16944.48,46.903910,1673135939999,7.947772e+05,1547,21.757880,3.686838e+05,0


In [65]:
historical_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 975406 entries, 0 to 975405
Data columns (total 12 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   open_time                     975406 non-null  int64  
 1   open                          975406 non-null  float64
 2   high                          975406 non-null  float64
 3   low                           975406 non-null  float64
 4   close                         975406 non-null  float64
 5   volume                        975406 non-null  float64
 6   close_time                    975406 non-null  int64  
 7   quote_asset_volume            975406 non-null  float64
 8   num_trades                    975406 non-null  int64  
 9   taker_buy_base_asset_volume   975406 non-null  float64
 10  taker_buy_quote_asset_volume  975406 non-null  float64
 11  ignore                        975406 non-null  int64  
dtypes: float64(8), int64(4)
memory usage: 89.3 M

In [64]:
historical_df.to_csv(BINANCE_HISTORICAL_DF_PATH, index=False)

In [ ]:
# TODO: I should be able to run the real-time API for the present day and get the remaining real-time data to append to the historical_df